# TODO
#### Klassen protecten
```
import AutoMLWrapper.automlwrapper as aw
aw.Configuration.Configuration
```

#### Erweiterung um Bibliotheken
-  @__library.setter
    - a) Global/GlobalConfig.yaml
        - lib-names
        - type mappings
    - b) directory durchsuchen 
- import in AutoMLWrapper anpassen
- SEDAR automl.py config locations anpassen

### custom proprocessing
- typen checken
- die wrapper könnten property methods implementieren, die die Rückgabe eines festen samples prüfen

### custom parameters


### 2+ problem types


## mlflow
- keras: Netzwerkschichten, Artifact Bild?
- Inferenz

In [1]:
import io
import mlflow
import pandas as pd
import numpy as np
from PIL import Image

from keras.datasets import mnist

#from automlwrapper import AutoMLWrapper
from AutoMLWrapper.automlwrapper import AutoMLWrapper

RUN_ALL = 0
UPLOAD_ALL = 0

RUN_KERAS = 1
UPLOAD_KERAS = 1

RUN_SKLEARN = 0
UPLOAD_SKLEARN = 0

RUU_GLUON = 1
UPLOAD_GLUON = 1

2023-12-06 11:57:06.352201: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-06 11:57:06.434982: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-12-06 11:57:06.434998: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-12-06 11:57:06.906388: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

Using TensorFlow backend


In [2]:
# #experiment_id = mlflow.create_experiment('test_automl', artifact_location='/home/mibbels/sedar-masterarbeit/sedar/sedar/automl/tmp/artifacts')

experiment_id = 5
remote_server_uri = "http://127.0.0.1:6798"
mlflow.set_tracking_uri(remote_server_uri) 
mlflow.set_experiment(experiment_id=experiment_id)

<Experiment: artifact_location='/home/mibbels/sedar-masterarbeit/sedar/sedar/automl/tmp/artifacts', creation_time=1701189486674, experiment_id='5', last_update_time=1701189486674, lifecycle_stage='active', name='test_automl', tags={}>

# Bilddaten

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x = np.concatenate((x_train, x_test))
y = np.concatenate((y_train, y_test))

In [4]:
from autogluon.multimodal.utils.misc import shopee_dataset
download_dir = './data/ag_automm_tutorial_imgcls'
shopee_train, shopee_test = shopee_dataset(download_dir, is_bytearray=True)

100%|██████████| 41.9M/41.9M [00:06<00:00, 6.52MiB/s]
Unzipping ./data/ag_automm_tutorial_imgcls/file.zip to ./data/ag_automm_tutorial_imgcls
INFO:root:Extract files...


In [5]:
def save_jpg_format(img):
    img_2d = img.reshape(int(np.sqrt(img.shape[0])), -1)
    img_pil = Image.fromarray(img_2d)
    buffer = io.BytesIO()
    img_pil.save(buffer, format='JPEG')
    img_bytes = bytearray(buffer.getvalue())
    return img_bytes

flattened_images = []
for img in x_train.reshape(x_train.shape[0], -1):
    img_bytes = save_jpg_format(img)
    flattened_images.append([img_bytes])
    
df_mnist = pd.DataFrame(list(zip(flattened_images, y_train)), columns=['image', 'label'])

# Tabellendaten

In [6]:
df_glass = pd.read_csv('data/glass.csv')


# Zeitreihen

In [7]:
df_m4 = pd.read_csv("https://autogluon.s3.amazonaws.com/datasets/timeseries/m4_hourly_subset/train.csv")


In [8]:
df_m4

,item_id,timestamp,target
0,H1,1750-01-01 00:00:00,605.0
1,H1,1750-01-01 01:00:00,586.0
2,H1,1750-01-01 02:00:00,586.0
3,H1,1750-01-01 03:00:00,559.0
4,H1,1750-01-01 04:00:00,511.0
...,...,...,...
148055,H200,1750-02-09 19:00:00,24.2
148056,H200,1750-02-09 20:00:00,22.6
148057,H200,1750-02-09 21:00:00,20.8
148058,H200,1750-02-09 22:00:00,19.6


In [9]:
user_hp = {'epochs':20, 'time_limit': 240, 'memory_limit':7000, 'testtest': 123}#, 'num_trials': 4,}


leaderboard(detailed=True) -> dataframe
- type: random_forest
- cost: validation loss
- duration: ?
- data_preprocessoers : name sklearn class?
. feature_preprocessors : name sklearn class


show_models()
- model_id, rank, cost
- feature_preprocessor: autosklearn object
- data_preprocessor: autosklearn object
 
Alle Parameter res[3]['sklearn_classifier'].get_params()

Modell res[3]['sklearn_classifier']


In [10]:
ask = AutoMLWrapper('autosklearn')
ask.SetOutputDirectory('test/ask/2')

if RUN_ALL or RUN_SKLEARN:
    ask.Train(data=df_glass, target_column='Type',
        task_type='classification',
        data_type='tabular',
        problem_type='multiclass',
        hyperparameters=user_hp
    )


if UPLOAD_ALL or UPLOAD_SKLEARN:
    print(ask.Output())
    ask.MlflowUploadBest({})

## multimodal
content in "fit(save_path)"
    - asset.json - //autogluon info
    - config.yaml model+hp
    - hparams.yaml
    zip + pkl files

fit_summary()
    {'val_accuracy': 0.956250011920929, 'training_time': 314.0019574165344}

hpo
    ... !!!

## tabular
content in TabularPredictor(save_path)
    - pkl files
    
fit_summary()
    - performance (kein direktes ranking)
    - hp
    
## timeseries
content in TimeSeriesPredictor(save_path)
    - pkl files

leaderboard()
    - model, score

fit_summary()
    - performance, path, time

info()
    






In [11]:
agl = AutoMLWrapper('autogluon')
agl.SetOutputDirectory('test/agl/1')

if RUN_ALL or RUU_GLUON: 
    agl.Train(
        data=df_m4,
        target_column='target',
        task_type='forecast',
        data_type='timeseries',
        problem_type='forecast',
        hyperparameters=user_hp
   )

================ TimeSeriesPredictor ================
TimeSeriesPredictor.fit() called
Fitting with arguments:
{'enable_ensemble': True,
 'evaluation_metric': None,
 'excluded_model_types': None,
 'hyperparameter_tune_kwargs': None,
 'hyperparameters': 'default',
 'num_val_windows': 1,
 'prediction_length': 1,
 'random_seed': None,
 'target': 'target',
 'time_limit': 240,
 'verbosity': 2}
Provided training data set with 148060 rows, 200 items (item = single time series). Average time series length is 740.3. Data frequency is 'H'.
AutoGluon will save models to test/agl/1/
AutoGluon will gauge predictive performance using evaluation metric: 'mean_wQuantileLoss'
	This metric's sign has been flipped to adhere to being 'higher is better'. The reported score can be multiplied by -1 to get the metric value.

Provided dataset contains following columns:
	target:           'target'

Starting training. Start time is 2023-12-06 11:57:57
Models that will be trained: ['Naive', 'SeasonalNaive', 'The

In [12]:
if UPLOAD_ALL or UPLOAD_GLUON:
    print(agl.Output())
    agl.MlflowUploadBest({})

/home/max/miniconda3/envs/AutoML/lib/python3.8/site-packages/_distutils_hack/__init__.py:36: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│   1 if UPLOAD_ALL or UPLOAD_GLUON:                                                               │
│   2 │   print(agl.Output())                                                                      │
│ ❱ 3 │   agl.MlflowUploadBest({})                                                                 │
│   4                                                                                              │
│                                                                                                  │
│ /home/max/Documents/Masterarbeit/AutoML/AutoMLWrapper/automlwrapper/AutoMLWrapper.py:150 in      │
│ MlflowUploadBest                                                                                 │
│                                                                                                  │
│   147 │   │   │   return                                                                         │
│   148 │   │                                                                                      │
│   149 │   │   mlflow_handler = MLflowHandler(best_model_info, user_tags, self.__out_path)        │
│ ❱ 150 │   │   mlflow_handler.log_to_mlflow()                                                     │
│   151 │                                                                                          │
│   152 │   #-----------------------------------------------------------------------------------   │
│   153 │   def MlflowUploadTopN(self, n: int, user_tags: dict):                                   │
│                                                                                                  │
│ /home/max/Documents/Masterarbeit/AutoML/AutoMLWrapper/automlwrapper/MLflowHandler.py:30 in       │
│ log_to_mlflow                                                                                    │
│                                                                                                  │
│   27 │   │   │   for key, value in model_info_dict['images'].items():                            │
│   28 │   │   │   │   mlflow.log_image(value, key)                                                │
│   29 │   │   │                                                                                   │
│ ❱ 30 │   │   │   mlflow.pyfunc.log_model("model", python_model=self.model_info.model_object)     │
│   31                                                                                             │
│                                                                                                  │
│ /home/max/miniconda3/envs/AutoML/lib/python3.8/site-packages/mlflow/pyfunc/__init__.py:1459 in   │
│ log_model                                                                                        │
│                                                                                                  │
│   1456 │   :return: A :py:class:`ModelInfo <mlflow.models.model.ModelInfo>` instance that conta  │
│   1457 │   │   │    metadata of the logged model.                                                │
│   1458 │   """                                                                                   │
│ ❱ 1459 │   return Model.log(                                                                     │
│   1460 │   │   artifact_path=artifact_path,                                                      │
│   1461 │   │   flavor=mlflow.pyfunc,                                                             │
│   1462 │   │   loader_module=loader_module,                                                      │
│                                                                                                  │
│ /home/max/miniconda3/envs/AutoML/lib/python3.8/site-package

In [ ]:
akr = AutoMLWrapper('autokeras')
akr.SetOutputDirectory('test/akr/1')

if RUN_ALL or RUN_KERAS:
    akr.Train(data=df_mnist, target_column='Type',
        task_type='classification',
        data_type='image',
        problem_type='multiclass',
        hyperparameters=user_hp
    )

if UPLOAD_ALL or UPLOAD_KERAS:
    print(akr.Output())
    akr.MlflowUploadBest({})

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:5                                                                                    │
│                                                                                                  │
│    2 akr.SetOutputDirectory('test/akr/1')                                                        │
│    3                                                                                             │
│    4 if RUN_ALL or RUN_KERAS:                                                                    │
│ ❱  5 │   akr.Train(data=mnist, target_column='Type',                                             │
│    6 │   │   task_type='classification',                                                         │
│    7 │   │   data_type='image',                                                                  │
│    8 │   │   problem_type='multiclass',                                                          │
│                                                                                                  │
│ /home/mibbels/sedar-masterarbeit/sedar/sedar/automl/AutoMLWrapper/automlwrapper/AutoMLWrapper.py │
│ :123 in Train                                                                                    │
│                                                                                                  │
│   120 │   │   if not self.__is_initialized:                                                      │
│   121 │   │   │   self.Initialize(data, target_column, task_type, data_type, problem_type)       │
│   122 │   │                                                                                      │
│ ❱ 123 │   │   self.__library._train_model(data, target_column, hyperparameters)                  │
│   124 │                                                                                          │
│   125 │   #-----------------------------------------------------------------------------------   │
│   126 │   def Evaluate(self, test_data: Any) -> float:                                           │
│                                                                                                  │
│ /home/mibbels/sedar-masterarbeit/sedar/sedar/automl/AutoMLWrapper/automlwrapper/AutoKeras/AutoKe │
│ rasWrapper.py:48 in _train_model                                                                 │
│                                                                                                  │
│    45 │   │   elif self.data_type == 'tabular':                                                  │
│    46 │   │   │   self._train_structured(data, target_column, user_hyperparameters)              │
│    47 │   │   elif self.data_type == 'image':                                                    │
│ ❱  48 │   │   │   self._train_image(data, target_column, user_hyperparameters)                   │
│    49 │                                                                                          │
│    50 │                                                                                          │
│    51 │   def _get_params(self, func_type: str, model_type: str):                                │
│                                                                                                  │
│ /home/mibbels/sedar-masterarbeit/sedar/sedar/automl/AutoMLWrapper/automlwrapper/AutoKeras/AutoKe │
│ rasWrapper.py:95 in _train_image                                                                 │
│                                                                                                  │
│    92 │   │   │   self.log_model_type = self.config._get_mlflow_details('ImageRegressor').get(   │
│    93 │   │                                                                                      │
│    94 │   │   self.fit_output = self.model.fit(                                                  │
│ ❱  95 │   │   │   **(self.data_preprocessing(data, target_c

# MLFlow
~/miniconda3/envs/automl_env/bin/pip

In [ ]:
df = df_glass.drop(columns=['Type'])
df

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.00,0.0
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.00,0.0
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.00,0.0
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.00,0.0
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.00,0.0
...,...,...,...,...,...,...,...,...,...
209,1.51623,14.14,0.00,2.88,72.61,0.08,9.18,1.06,0.0
210,1.51685,14.92,0.00,1.99,73.06,0.00,8.40,1.59,0.0
211,1.52065,14.36,0.00,2.02,73.42,0.00,8.44,1.64,0.0
212,1.51651,14.38,0.00,1.94,73.61,0.00,8.48,1.57,0.0


In [ ]:
logged_model = 'runs:/a0337b9ecfe84e99b59ff3a9639f1ef6/model'
loaded_model = mlflow.pyfunc.load_model(logged_model)

#loaded_model.predict(test_data_byte)